# GPT4o API로 2023 수능 국어 풀어보기

**시간이 부족하여 구현을 못했으나, 진행한 부분까지만 임시 제출합니다.🫠**

1. 데이터 파악
2. 테스트 설계
- 1) 데이터를 어떻게 가공할 것인가?
    - 1-1. 지문 + 꼬리 문제 형태로 가공한다.
    - 1-2. 각 꼬리 문제의 5개 선지 시작 부분에 번호를 추가한다('1.' + 기존 선지내용).
- 2) 프롬프트를 어떻게 넣을 것인가?
    - 2-1. Parsing을 위한 프롬프팅
        - 추론 과정 요구 프롬프트 + 정답 요구 프롬프트        
        - <solve>number</solve> 로 요구한다. (예를 들어 <solve>3</solve>)
        - 이해도를 높이기 위해 one-shot prompting을 적용한다.
    - 2-2. 정답률을 높이기 위한 프롬프팅
        - 문제당 두 번의 문제 풀이를 독립적으로 진행한다.
        - 한 번은 zero-shot-CoT, 또 한 번은 plan-and-solve 를 적용하여 논리적 추론을 유도한다.
        - CoT 와 plan-and-solve에서 최종 도출한 정답이 동일할 경우 이를 제출하는 것으로 간주한다.
        - 정답이 다를 경우, 각 추론 기록을 참고하여 마지막 결정을 하는 추가 문제 풀이를 한다.
- 3) 정확도를 어떻게 측정할 것인가?
    - 문제 풀이마다 즉시 채점을 한다. 오답일 경우에 감점(minus) 변수에 가점한다. 마지막에 100 - minus 으로 점수를 도출한다.
    - 복기를 위해, 오답일 경우 wrong_solves 리스트에 question, solve (모델 답변), answer (실제 정답), score를 저장한다.
3. 구현
- 하나의 문제에 대해서 GPT-4의 예측 결과를 내놓는 함수를 `def prediction(problem)`이라는 signature로 만드셔야 합니다. `problem`은 json 형태의 문제입니다.
- `def prediction` 함수를 모든 수능 국어 문제들에 대해서 돌린 후, 실제 정답과 비교하여 GPT-4의 점수를 계산하는 코드를 구현하시면 됩니다.

# [MY CODE] 데이터 파악
간단한 데이터로 진행하는 실습이므로, 별도의 파일에 준비한 json 데이터를 직접 참조하여 진행합니다.

[데이터 설명]
- id: 문제 지문의 id
- pragraph: 문제 지문
- type: 지문 타입 (화법과 작문, 문학, 비문학 등..)
- problems: 문제 정보들
    - question: 질문
    - choices: 보기(5개)
    - answer: 정답
    - score: 배점

In [36]:
import data

print(data.kice_data_2023[0],'\n')
print(data.kice_data_2023[0].keys(),'\n')
print(data.kice_data_2023[0]['problems'][0].keys())

{'id': '2023_11_KICE_1-3', 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통

# [MY CODE] 데이터 가공
- 1) 데이터를 어떻게 가공할 것인가?
    - 1-1. 지문 + 꼬리 문제 형태로 가공한다.
        - 예를 들어 지문 하나에 꼬리 문제 2개라면, '지문 + 꼬리 문제1 + 보기 5개', '지문 + 꼬리 문제2 + 보기 5개'로 각각 나눈다.
    - 1-2. choices의 각 선지 첫 부분에 번호를 추가한다('1.' + 기존지문).

In [33]:
processed_data = []

# 데이터 가공
for idx, entry in enumerate(data.kice_data_2023, start=1):
    for problem in entry["problems"]:
        choices = []
        for i, choice in enumerate(problem["choices"], start=1):
            choices.append(f"[{i}] {choice}")

    # 새로운 항목 생성
    processed_entry = {
        "id": entry["id"],
        "type": entry["type"],
        "paragraph": entry["paragraph"],
        "question": problem["question"],
        "choices": choices,
        "answer": problem["answer"],
        "score": problem["score"]
    }
    # 결과 리스트에 추가
    processed_data.append(processed_entry)

# 결과 출력
for key, value in processed_data[0].items():
    print(f"{key}: {value}")


id: 2023_11_KICE_1-3
type: 0
paragraph: 사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통한

# [MY CODE] 프롬프트 준비

In [ ]:
# GPT-4 예측 함수
def prediction(problem):
    """
    문제를 JSON 형식으로 받아 GPT-4를 통해 정답을 예측하는 함수.
    문제 풀이는 Zero-shot-CoT 및 Plan-and-solve 방식을 사용.
    """
    question = problem["question"]
    choices = problem["choices"]
    answer = problem["answer"]  # 실제 정답 (채점에 사용)

    # 프롬프트 생성
    paragraph = problem.get("paragraph", "No paragraph provided.")
    user_prompt = (
        f"Given the following passage:\n\n{paragraph}\n\n"
        f"Question: {question}\n\n"
        f"Choices:\n"
        + "\n".join([f"[{i+1}] {choice}" for i, choice in enumerate(choices)])
        + "\n\n"
        "추론 과정:\n"
        "Please provide a detailed, step-by-step reasoning process for your answer.\n\n"
        "정답 도출:\n"
        "Provide the final answer in the format <solve>number</solve>. (e.g., <solve>1</solve>)"
    )

    # OpenAI API 요청
    response_cot = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": assistant_role},
            {"role": "user", "content": user_prompt},
        ],
        n=1,
        max_tokens=4096,
        temperature=0.3,
    )
    cot_result = response_cot.choices[0].message["content"].strip()

    # Plan-and-solve 방식 프롬프트
    user_prompt_plan = (
        f"Given the following passage:\n\n{paragraph}\n\n"
        f"Question: {question}\n\n"
        f"Choices:\n"
        + "\n".join([f"[{i+1}] {choice}" for i, choice in enumerate(choices)])
        + "\n\n"
        "Plan:\n"
        "First, provide a short plan to approach this question.\n\n"
        "Solve:\n"
        "Based on the plan, provide the final answer in the format <solve>number</solve>. (e.g., <solve>1</solve>)"
    )

    response_plan = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": assistant_role},
            {"role": "user", "content": user_prompt_plan},
        ],
        n=1,
        max_tokens=512,
        temperature=0.7,
    )
    plan_result = response_plan.choices[0].message["content"].strip()

    # 정답 파싱
    def extract_answer(response):
        import re
        match = re.search(r"<solve>(\d)</solve>", response)
        return int(match.group(1)) if match else None

    cot_answer = extract_answer(cot_result)
    plan_answer = extract_answer(plan_result)

    # 결과 처리
    if cot_answer == plan_answer:
        final_answer = cot_answer
    else:
        # 추가 풀이 요청 (모델 간 불일치 시)
        additional_prompt = (
            f"Previous answers were not unanimous.\n\n"
            f"Passage: {paragraph}\n\n"
            f"Question: {question}\n\n"
            f"Choices:\n"
            + "\n".join([f"[{i+1}] {choice}" for i, choice in enumerate(choices)])
            + "\n\n"
            "추론 과정:\n"
            "Reconsider your previous answers and provide a detailed reasoning process.\n\n"
            "정답 도출:\n"
            "Provide the final answer in the format <solve>number</solve>."
        )
        additional_response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": assistant_role},
                {"role": "user", "content": additional_prompt},
            ],
            n=1,
            max_tokens=512,
            temperature=0.7,
        )
        final_answer = extract_answer(additional_response.choices[0].message["content"])

    # 채점
    correct = final_answer == answer
    return {
        "question": question,
        "choices": choices,
        "model_answer": final_answer,
        "correct_answer": answer,
        "score": problem["score"],
        "correct": correct,
    }

# 전체 데이터 처리 및 점수 계산
def evaluate_model(problems):
    results = []
    total_score = 0
    minus_score = 0

    for problem in problems:
        result = prediction(problem)
        results.append(result)
        if result["correct"]:
            total_score += result["score"]
        else:
            minus_score += result["score"]

    # 최종 점수
    accuracy = 100 - minus_score
    return {"results": results, "total_score": total_score, "accuracy": accuracy}


In [ ]:
# from openai import OpenAI
# from google.colab import userdata
# import api_key

# open_ai_api_key = api_key.open_ai_api_key
# client = OpenAI(api_key=userdata.get(open_ai_api_key))

# temperature = 0.7  # temperature 값이 높이고 n을 5로 설정하여 다양한 범위의 답을 생성한다.
# max_tokens = 4096  # 생성하는 최대 token 개수
# n = 5  # 같은 질의에 대해 몇 개의 답변을 출력할지 결정
# # frequency_penalty = 0.0  # 같은 단어가 반복적으로 나오는 것을 방지하기 위한 옵션

# assistant_role = (
#     "You are a top-performing student in the Korean CSAT (수능) exam for the Korean subject. "
#     "You solve problems logically and step by step, providing a detailed reasoning process "
#     "followed by the final answer in the format <answer>정답</answer>. "
#     "Always include both '추론 과정' and '정답 도출'."
# )

# def ask_model(problem, paragraph, n=5):
#     question = problem["question"]
#     choices = problem["choices"]

#     # User Prompt 구성
#     user_prompt = (
#         f"Given the following passage:\n\n{paragraph}\n\n"
#         f"Question: {question}\n\n"
#         "Choices:\n"
#         + "\n".join([f"[{i+1}] {choice}" for i, choice in enumerate(choices)])
#         + "\n\n"
#         "추론 과정:\n"
#         "Please provide a detailed reasoning process for your answer.\n\n"
#         "정답 도출:\n"
#         "Provide the final answer in the format <answer>정답</answer>."
#     )

#     # OpenAI API 요청
#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": assistant_role},
#             {"role": "user", "content": user_prompt},
#         ],
#         n=n,
#         max_tokens=max_tokens,
#         temperature=temperature,
#     )

#     # 응답 수집
#     return [choice["message"]["content"].strip() for choice in response.choices]


In [ ]:
# message=[{"role": "user", "content": "Is the following movie review positive or negative?: " + imdb["test"][123]["text"]}]
# response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=message,
#     n=5,
#     max_tokens=max_tokens,
#     temperature=temperature,
#     frequency_penalty=frequency_penalty
# )

# print(imdb["test"][123]["label"])
# for i in range(5):
#     print(response.choices[i].message.content)

In [ ]:
# prompt = "Is the following movie review positive or negative?\n\n"
# for i in range(10):
#     text = imdb["train"][i]["text"]
#     label = imdb["train"][i]["label"]

#     sub_prompt = "Review: " + text
#     if label == 0:
#         sub_prompt += "\nAnswer: negative\n\n"
#     else:
#         sub_prompt += "\nAnswer: positive\n\n"
#     prompt += sub_prompt

# prompt += "Review: "
# print(prompt)